# Mount Google Drive
since I am using google colab pro for developing the code, mounting to my google drive is needed

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


Set root folder

In [2]:
root = '/gdrive/My Drive/MSB481/'

# Import Libraries

In [3]:
import numpy as np 
import os
import tensorflow as tf
from tensorflow.keras.models import load_model

# Load the trained model

In [5]:
loaded_model = load_model(os.path.join(root, 'model/best_150_new_dataset.h5'))
loaded_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        208       
                                                                 
 activation (Activation)     (None, 96, 96, 16)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        2080      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 24, 24, 32)       0         
 2D)                                                    

# Converting process
I use the code from the class tutorial. However, remember that to change the channel from greyscale (1 channel) to RGB (3 channels)

In [8]:
WIDTH = 96
HEIGHT = 96

def representative_dataset():
    for _ in range(100):
      data = np.random.rand(1, WIDTH, HEIGHT, 3)
      yield [data.astype(np.float32)]
      
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE] #[tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.int8  # or tf.uint8
# converter.inference_output_type = tf.int8  # or tf.uint8
tflite_quant_model = converter.convert()
file = open(os.path.join(root, 'model/converted_model_best_150_new_dataset.tflite') , 'wb') 
file.write(tflite_quant_model) 

/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


59768

convert from tflite to c++ code

In [10]:
!xxd -i "/gdrive/My Drive/MSB481/model/converted_model_best_150_new_dataset.tflite" > "/gdrive/My Drive/MSB481/model/model_data.tflite"